In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
# Importa la función para crear la matriz de confusión
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')
#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
#df_jo = df_jo.drop(columns=['index'])
#df_jo = df_jo.drop(columns=['Mes_created_at'])
#df_jo_cp = df_jo.copy()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])
#df_jo.info()


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f40f5806d10>>
Traceback (most recent call last):
  File "/home/albagd/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [3]:
df_jo.columns

Index(['id_cr', 'id_fe', 'fe_cr_id', 'user_id', 'active', 'created_at',
       'created_at_fe', 'amount', 'fee', 'stat_cr', 'stat_fe', 'transfer_type',
       'type', 'to_receive_ini', 'to_receive_bank', 'to_reimbur',
       'to_reimbur_cash', 'to_end', 'to_send', 'send_at', 'cr_received_date',
       'money_back_date', 'reimbursement_date', 'paid_at', 'charge_moment',
       'moderated_at', 'reason', 'category', 'from_date', 'to_date',
       'recovery_status', 'updated_at', 'reco_creation', 'reco_last_update',
       'updated_at_fe', 'Mes_created_at', 'cash_request_received_date',
       'created_at_slot', 'created_at_slot_h', 'created_at_dow',
       'needs_m_check', 'needs_m_check_recov', 'n_fees', 'n_backs',
       'n_recovery', 'n_incidents', 'created_at_d', 'GBP_EUR', 'BTC_GBP',
       'inflation', 'unemploy_rate'],
      dtype='object')

In [4]:
X_log_L=df_jo.drop(columns=['id_cr','id_fe','fe_cr_id','active','stat_cr','stat_fe','recovery_status','needs_m_check','needs_m_check_recov','Mes_created_at','created_at_slot_h','created_at_d'])
y_log_L=df_jo['needs_m_check_recov']

In [ ]:
X_log_L.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32092 entries, 0 to 32091
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   user_id                     32092 non-null  int64          
 1   created_at                  32092 non-null  datetime64[ns] 
 2   created_at_fe               21055 non-null  datetime64[ns] 
 3   amount                      32092 non-null  float64        
 4   fee                         32092 non-null  float64        
 5   transfer_type               32092 non-null  object         
 6   type                        32092 non-null  object         
 7   to_receive_ini              25366 non-null  timedelta64[ns]
 8   to_receive_bank             18691 non-null  timedelta64[ns]
 9   to_reimbur                  32092 non-null  timedelta64[ns]
 10  to_reimbur_cash             22677 non-null  timedelta64[ns]
 11  to_end                      24106 non-nul

: 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X= X_log_L
y= y_log_L


# Convertir las columnas de tipo timedelta64
for col in X.select_dtypes(include=['timedelta64']).columns:
    X[col] = X[col].apply(lambda x: x.total_seconds() if pd.notnull(x) else None)

# One Hot econding para pasar a numéricas todas las categóricas
X = pd.get_dummies(X, columns=['transfer_type','type','charge_moment','reason','category'], drop_first=False, dtype=int)

# X son las características y y es la variable objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos (importante para regularización)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo con penalización L1 (similar a Lasso)
logreg_l1 = LogisticRegression(penalty='l1', solver='saga', C=1.0, random_state=42) #probar tb con solver='liblinear'
logreg_l1.fit(X_train_scaled, y_train)

# Modelo con penalización L2 (similar a Ridge)
logreg_l2 = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, random_state=42)
logreg_l2.fit(X_train_scaled, y_train)

# Predicciones
y_pred_l1 = logreg_l1.predict(X_test_scaled)
y_pred_l2 = logreg_l2.predict(X_test_scaled)

# Evaluación (por ejemplo, exactitud)
from sklearn.metrics import accuracy_score
print("Exactitud L1:", accuracy_score(y_test, y_pred_l1))
print("Exactitud L2:", accuracy_score(y_test, y_pred_l2))


In [ ]:
# Extraer los coeficientes para la L1
coefs = logreg_l1.coef_[0]  # Para regresión logística binaria, es un vector 1D

# Crear un DataFrame para visualizar los coeficientes junto con las características
importancia = pd.DataFrame({
    'Característica': X_train.columns,
    'Coeficiente': coefs
})

# Ordenar por la magnitud de los coeficientes
importancia = importancia.reindex(importancia['Coeficiente'].abs().sort_values(ascending=False).index)

# Mostrar las características con coeficientes no nulos (para Lasso)
importancia_no_nula = importancia[importancia['Coeficiente'] != 0]

print("Características importantes:")
print(importancia_no_nula)

In [ ]:
# Extraer los coeficientes para la L2
coefs2 = logreg_l2.coef_[0]  # Para regresión logística binaria, es un vector 1D

# Crear un DataFrame para visualizar los coeficientes junto con las características
importancia2 = pd.DataFrame({
    'Característica': X_train.columns,
    'Coeficiente': coefs2
})

# Ordenar por la magnitud de los coeficientes
importancia2 = importancia2.reindex(importancia2['Coeficiente'].abs().sort_values(ascending=False).index)

# Mostrar las características con coeficientes no nulos (para Lasso)
importancia_no_nula2 = importancia2[importancia2['Coeficiente'] != 0]

print("Características importantes:")
print(importancia_no_nula2)

NameError: name 'logreg_l2' is not defined